# Copied from CatBoost_14Feature_Cross_Validation

Almost no work by me - just a bit of FE.

see https://www.kaggle.com/andy6804tw/catboost-13feature-cross-validation

Removed the time feature: all_data['date_time'].astype(np.int64)//10**9

Added some lags and fourier features (to try to pick up the season component).


In [ ]:
import numpy as np
import matplotlib.pyplot as plt 
import pandas as pd  
import seaborn as sns 
from sklearn.datasets import load_boston
import warnings
warnings.filterwarnings('ignore')
plt.style.use('ggplot')

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-jul-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-jul-2021/test.csv')
print('train shape:',train.shape)
print('test shape:',test.shape)

In [ ]:
# found that one of the test data is in the train set and removed it
# Dropping the last row which is 2011-01-01 00:00:00
train=train.loc[~(train['date_time']=='2011-01-01 00:00:00')].reset_index(drop=True)

In [ ]:
print('train shape:',train.shape)

In [ ]:
all_data = pd.concat([train, test])
# convert to datatime format
all_data['date_time'] = pd.to_datetime(all_data['date_time'])
all_data.head()

In [ ]:
all_data.isnull().sum()

In [ ]:
import math

def pb_add(X):
    X['day'] = X.date_time.dt.weekday
    is_odd = (X['sensor_4'] < 646) & (X['absolute_humidity'] < 0.238)
    X['is_odd'] = is_odd
    diff = X['date_time'] - min(X['date_time'])
    trend = diff.dt.days
    X['f1s'] = np.sin(trend * 2 * math.pi / (365 * 1)) 
    X['f1c'] = np.cos(trend * 2 * math.pi / (365 * 1))
    X['f2s'] = np.sin(2 * math.pi * trend / (365 * 2)) 
    X['f2c'] = np.cos(2 * math.pi * trend / (365 * 2)) 
    X['f3s'] = np.sin(2 * math.pi * trend / (365 * 3)) 
    X['f3c'] = np.cos(2 * math.pi * trend / (365 * 3)) 
    X['f4s'] = np.sin(2 * math.pi * trend / (365 * 4)) 
    X['f4c'] = np.cos(2 * math.pi * trend / (365 * 4)) 
    X['fh1s'] = np.sin(diff.dt.seconds * 2 * math.pi / ( 3600 * 24 * 1))
    X['fh1c'] = np.cos(diff.dt.seconds * 2 * math.pi / ( 3600 * 24 * 1))
    X['fh2s'] = np.sin(diff.dt.seconds * 2 * math.pi / ( 3600 * 24 * 2))
    X['fh2c'] = np.cos(diff.dt.seconds * 2 * math.pi / ( 3600 * 24 * 2))
    X['fh3s'] = np.sin(diff.dt.seconds * 2 * math.pi / ( 3600 * 24 * 3))
    X['fh3c'] = np.cos(diff.dt.seconds * 2 * math.pi / ( 3600 * 24 * 3))
    
    sensor_features = [
        'deg_C', 
        'relative_humidity', 'absolute_humidity', 
        'sensor_1', 'sensor_2', 'sensor_3', 'sensor_4', 'sensor_5' ]
    
    lags = [-1, -4, -24, -7 * 24]  # last 4 hours, last 2 days, last 4 weeks
    for sensor_feature in sensor_features:
        this = X[sensor_feature]
        # look back
        for lag in lags:
            feature = f'{sensor_feature}_{abs(lag)}b'
            this_f = X[sensor_feature].shift(lag)
            X[feature] = (this_f - this).fillna(0)
        # look forwards
        for lag in lags:
            feature = f'{sensor_feature}_{abs(-lag)}f'
            this_f = X[sensor_feature].shift(-lag)
            X[feature] = (this_f - this).fillna(0)
            
    return X

In [ ]:
# The months will be used for folds split
months = all_data["date_time"].dt.month[:len(train)]
## New idea
all_data["hour"] = all_data["date_time"].dt.hour
all_data["working_hours"] =  all_data["hour"].isin(np.arange(8, 21, 1)).astype("int")
all_data["is_weekend"] = (all_data["date_time"].dt.dayofweek >= 5).astype("int")
all_data['hr'] = all_data.date_time.dt.hour*60+all_data.date_time.dt.minute
all_data['satday'] = (all_data.date_time.dt.weekday==5).astype("int")
all_data["SMC"] = (all_data["absolute_humidity"] * 100) / all_data["relative_humidity"]
all_data.drop(columns = 'hour', inplace = True)


# convert datetime to timestamp(s)
# included::: all_data['time'] = all_data['date_time'].astype(np.int64)//10**9

all_data = pb_add(all_data.copy())

all_data.drop(columns = 'date_time', inplace = True)

In [ ]:
X=all_data[:len(train)].drop(columns = ['target_carbon_monoxide', 'target_benzene', 'target_nitrogen_oxides'])
y=all_data[:len(train)][['target_carbon_monoxide', 'target_benzene', 'target_nitrogen_oxides']]
y_log=np.log(y)
X_test=all_data[len(train):].drop(columns = ['target_carbon_monoxide', 'target_benzene', 'target_nitrogen_oxides'])
print('X_train shape:', X.shape)
print('y_train shape:', y.shape)
print('X_test shape:', X_test.shape)

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(X)
X_scaled = scaler.transform(X)
X_test_scaled = scaler.transform(X_test)

In [ ]:
X_scaled=X.values
X_test_scaled=X_test.values

In [ ]:
X_scaled

## Train Model

In [ ]:
# load submission
preds = pd.read_csv('../input/tabular-playground-series-jul-2021/sample_submission.csv')

In [ ]:
# Sets of hyperparameters optimized by Optuna for each target
cb_params = [
                {'learning_rate': 0.010169009412219588,
                 'l2_leaf_reg': 8.908337085912136,
                 'bagging_temperature': 8.384477224270551,
                 'random_strength': 1.950237493637981,
                 'depth': 6,
                 'grow_policy': 'Lossguide',
                 'leaf_estimation_method': 'Newton'},
                {'learning_rate': 0.166394867169309,
                 'l2_leaf_reg': 8.704675157564441,
                 'bagging_temperature': 3.340826164726799,
                 'random_strength': 1.538518016574368,
                 'depth': 2,
                 'grow_policy': 'Depthwise',
                 'leaf_estimation_method': 'Newton'},
                {'learning_rate': 0.028141156076957437,
                 'l2_leaf_reg': 3.116523267336638,
                 'bagging_temperature': 4.420661209459851,
                 'random_strength': 1.8011752694610028,
                 'depth': 6,
                 'grow_policy': 'Depthwise',
                 'leaf_estimation_method': 'Newton'},
            ]

## LeaveOneGroupOut
cat_features=["working_hours","is_weekend","satday"] 8、9、11

Public Score: 
- 0.19336 with 13 feature 
 - ['deg_C', 'relative_humidity', 'absolute_humidity', 'sensor_1', 'sensor_2', 'sensor_3', 'sensor_4', 'sensor_5', 'working_hours', 'is_weekend', 'time']

inside test error:
- 0.18441005 with 13 features (0.14049546445212416+0.09160805296249219+0.3211266334764782)

In [ ]:
%%time
from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection import LeaveOneGroupOut
from catboost import CatBoostRegressor

all_fi = []
splits = 10
target_names=y_log.columns

for i, target in enumerate(target_names):
    print(f"\nTraining for {target}...")
    logo = LeaveOneGroupOut()
    oof_preds = np.zeros((X_scaled.shape[0],))
    model_preds = 0
    model_fi = 0
    for num, (train_idx, valid_idx) in enumerate(logo.split(X_scaled, y_log, months)):
        X_train, X_valid = X_scaled[[train_idx]], X_scaled[[valid_idx]]
        y_train, y_valid = y_log.loc[train_idx, target], y_log.loc[valid_idx, target]
        model = CatBoostRegressor(random_state=42,
                                 thread_count=4,
                                 verbose=False,
                                 loss_function='RMSE',
                                 eval_metric='RMSE',
                                 od_type="Iter",
                                 early_stopping_rounds=500,
                                 use_best_model=True,
                                 iterations=10000,
                                 **cb_params[i])
        model.fit(X_train, y_train,
                  eval_set=(X_valid, y_valid),
                  verbose=False)
        model_preds += np.exp(model.predict(X_test_scaled)) / splits
        model_fi += model.feature_importances_
        oof_preds[valid_idx] = np.exp(model.predict(X_valid))
        print(f"Fold {num} RMSLE: {np.sqrt(mean_squared_log_error(np.exp(y_valid), oof_preds[valid_idx]))}")
    print(f"\nOverall RMSLE: {np.sqrt(mean_squared_log_error(np.exp(y_log[target]), oof_preds))}")    
    preds[target] = model_preds
    all_fi.append(dict(zip(X.columns, model_fi)))

## original CV

* Training for target_carbon_monoxide...
* * Fold 0 RMSLE: 0.0778258850819034
* * Fold 1 RMSLE: 0.09567414611737445
* * Fold 2 RMSLE: 0.12458770150082436
* * Fold 3 RMSLE: 0.11865213752846315
* * Fold 4 RMSLE: 0.10702740862606132
* * Fold 5 RMSLE: 0.1022360313603644
* * Fold 6 RMSLE: 0.09396926069065796
* * Fold 7 RMSLE: 0.08563271122743246
* * Fold 8 RMSLE: 0.1750379107364064
* * Fold 9 RMSLE: 0.278176029653778

* Overall RMSLE: 0.1398755852324709

* Training for target_benzene...
* * Fold 0 RMSLE: 0.10108794639781213
* * Fold 1 RMSLE: 0.08710269182125585
* * Fold 2 RMSLE: 0.09114492935249197
* * Fold 3 RMSLE: 0.08179494264044823
* * Fold 4 RMSLE: 0.08300192021616676
* * Fold 5 RMSLE: 0.08387077658443987
* * Fold 6 RMSLE: 0.09327116316158779
* * Fold 7 RMSLE: 0.08067460865192715
* * Fold 8 RMSLE: 0.09538156034307499
* * Fold 9 RMSLE: 0.11808006284554567

* Overall RMSLE: 0.09190958831600804

* Training for target_nitrogen_oxides...
* * Fold 0 RMSLE: 0.18152157734387575
* * Fold 1 RMSLE: 0.2140913436184166
* * Fold 2 RMSLE: 0.19964470707304033
* * Fold 3 RMSLE: 0.24475069072351055
* * Fold 4 RMSLE: 0.2232802232637754
* * Fold 5 RMSLE: 0.49218598179725737
* * Fold 6 RMSLE: 0.4173263400059904
* * Fold 7 RMSLE: 0.2017441021227966
* * Fold 8 RMSLE: 0.2208801570446175
* * Fold 9 RMSLE: 0.5116902845744751

* Overall RMSLE: 0.3194398990409419

## Save Predict File

In [ ]:
preds.head()

In [ ]:
preds.to_csv('submission.csv', index=False)